In [5]:
import boto3
import sagemaker
import json

with open('credentials.json') as f:
    credentials = json.load(f)

ecr_namespace = "sagemaker-training-containers/"
prefix = "pyspark-ml"
role = credentials['role']
account_id = credentials['account_id']

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()
ecr_repository_name = ecr_namespace + prefix

In [6]:
region

'eu-central-1'

In [7]:
sagemaker_session

In [8]:
!#sudo docker build -f ./../docker/Dockerfile -t sagemaker-training-containers/basic-training-container ../docker

In [9]:
#%%capture
#! bash ./scripts/build_and_push.sh $account_id $region $ecr_repository_name

In [15]:
container_image_uri = "{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest".format(
    account_id, region, ecr_repository_name
)
print(container_image_uri)

855373717346.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-training-containers/pyspark-ml:latest


In [18]:
! echo "val1, val2, val3" > dummy.csv
print(sagemaker_session.upload_data("dummy.csv", bucket, prefix + "/train"))
print(sagemaker_session.upload_data("dummy.csv", bucket, prefix + "/val"))
! rm dummy.csv

s3://sagemaker-eu-central-1-855373717346/pyspark-ml/train/dummy.csv
s3://sagemaker-eu-central-1-855373717346/pyspark-ml/val/dummy.csv


In [19]:
import sagemaker

# Instances Pricing https://aws.amazon.com/sagemaker/pricing/
est = sagemaker.estimator.Estimator(
    container_image_uri,
    role,
    train_instance_count=1,
    #train_instance_type="local",  # use local mode
    train_instance_type='ml.m5.large',
    base_job_name=prefix,
)

est.set_hyperparameters(hp1="value1", hp2=300, hp3=0.001)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [20]:
train_config = sagemaker.session.s3_input(
    "s3://{0}/{1}/train/".format(bucket, prefix), content_type="text/csv"
)
val_config = sagemaker.session.s3_input(
    "s3://{0}/{1}/val/".format(bucket, prefix), content_type="text/csv"
)

est.fit({"train": train_config, "validation": val_config})

The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2022-08-07 09:33:31 Starting - Starting the training job...
2022-08-07 09:33:54 Starting - Preparing the instances for trainingProfilerReport-1659864799: InProgress
......
2022-08-07 09:35:35 Downloading - Downloading input data
2022-08-07 09:35:35 Training - Downloading the training image..Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/07 09:35:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable

2022-08-07 09:36:35 Uploading - Uploading generated training model
2022-08-07 09:36:35 Completed - Training job completed
Running training...
Hyperparameters configuration:
{'hp1': 'value1', 'hp2': '300', 'hp3': '0.001'}
Input data configuration:
{'train': {'ContentType': 'text/csv',
           'RecordWrapperType': 'None',
           'S3DistributionType': 'FullyReplicated',
           'TrainingInputMode': 'File'},
 'validation': {